In [ ]:
!pip install yahoo_fin
# !pip install --upgrade pandas-datareader

# !pip install --upgrade pandas
!pip install yfinance

#importing libraries

In [8]:
import pandas_datareader as web
import yfinance as yf
import pandas as pd
import numpy as np


#LOADING DATA AND STOCK SCREENING

In [22]:
path = '/nifty50list1.csv'
symbols= pd.read_csv(path)


Creating DataFrames

In [27]:
tickers = pd.DataFrame()
stockscreener_df =  pd.DataFrame()


MOVING AVERAGES 150 and 200

In [44]:
tickers['SMA_150'] = round(tickers['Close'].rolling(window=150).mean(),2)
tickers['SMA_200'] = round(tickers['Close'].rolling(window=200).mean(),2)

RSI Calculation

In [28]:
def calculate_rsi(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] 

        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi


Screening Process

In [ ]:
for sym in symbols['Symbol']:
  rsi=[]
  symbol = sym +'.NS'
  nifty50_df= yf.Ticker(symbol)
  tickers = nifty50_df.history(period="300D")
  tickers['SMA_150'] = round(tickers['Close'].rolling(window=150).mean(),2)
  tickers['SMA_200'] = round(tickers['Close'].rolling(window=200).mean(),2)
  prices = tickers['Close']
  rsi=calculate_rsi(prices)
  
  temp = tickers.tail(1)
  temp['RSI'] = rsi[-1]
  
  stockscreener_df = stockscreener_df.append(temp, ignore_index = True)
  stockscreener_df ['Symbol']= symbols['Symbol']

In [ ]:
stockscreener_df.drop(columns=['Dividends','Stock Splits'], inplace =True)
print(stockscreener_df)

Scanned Data




In [42]:
print('Long positions\n')
for i in range(stockscreener_df.shape[0]):
  condition1 = stockscreener_df['Open'][i]>stockscreener_df['SMA_150'][i] and stockscreener_df['SMA_150'][i]>stockscreener_df['SMA_200'][i] and 60>stockscreener_df['RSI'][i]>40
  if(condition1):
    print('{} '.format(stockscreener_df['Symbol'][i]))

Long positions

BAJAJFINSV 
BHARTIARTL 
BRITANNIA 
HDFCBANK 
HEROMOTOCO 
HDFC 
ICICIBANK 
INDUSINDBK 
JSWSTEEL 
LT 
M&M 
NTPC 
NESTLEIND 
SBILIFE 
SBIN 
SUNPHARMA 
TATASTEEL 
TITAN 
ULTRACEMCO 


In [43]:
print('Short positions\n')
for i in range(stockscreener_df.shape[0]):
  condition2  = stockscreener_df['Close'][i]<stockscreener_df['SMA_150'][i] and stockscreener_df['SMA_150'][i]<stockscreener_df['SMA_200'][i] and 60<=stockscreener_df['RSI'][i]
  if(condition2):
    print('{}'.format(stockscreener_df['Symbol'][i]))

Short positions

